In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from rdkit import Chem
from rdkit.Chem import inchi
from tqdm import tqdm
from time import sleep
from tqdm.notebook import tqdm

from chembl_structure_pipeline import standardizer as ChEMBL_standardizer
from papyrus_structure_pipeline import standardize

READ ME

This file is used to create the CNS database to validate the combined subMD, and subFP using models.



In [ ]:
#Load the raw drunkbank data
cns_raw =  pd.read_csv('DrugBank_parsed.tsv', delimiter= '\t')

In [ ]:
#Remove if no smiles is available

cns_smiles = cns_raw[((cns_raw['smiles'].notna()))]
cns_smiles = cns_smiles.reset_index(drop=True)
print(f'Length with SMILES: {len(cns_smiles)}')


In [ ]:
#Remove rows where categories is missing
cns_cat = cns_smiles[((cns_smiles['categories'].notna()))]
cns_cat = cns_cat.reset_index(drop=True)
print(f'Length without missing categories: {len(cns_cat)}')

In [ ]:
#Find CNS targeting drugs: create new column 'target'
keywords = ['CNS', 'Central Nervous System']
cns_cat['status_cns'] = cns_cat['categories'].apply(lambda x: 'CNS' if any(keyword.lower() in str(x).lower() for keyword in keywords) else 'Non-CNS')

In [ ]:
#Keep the relevant column

cns = cns_cat[['smiles','status_cns']]
print(len(cns))
cns.head()

cns = cns.rename(columns={'smiles': 'SMILES_raw'})


In [ ]:
#Papyrus Standardization

def create_sd_smiles(sd_mol):
    try:
        standardized_smiles =  Chem.MolToSmiles(sd_mol)
        return standardized_smiles
    except Exception as e:
        print(f"An sd_smiles error occurred: {str(e)}")
        return None
    
#Create InChI keys from standardized molecules
def mol_to_inchi_key(sd_mol):
    if sd_mol is not None:

        inchi_str = inchi.MolToInchi(sd_mol)
        inchi_key = inchi.InchiToInchiKey(inchi_str)
    else:
        inchi_key = None   
    return inchi_key

def standardize_molecule(mol):
    standardized_mol =  standardize(mol,raise_error=False )
    return standardized_mol

#Standardize 

def standardize_workflow(df_raw):
    for i in range(0,len(df_raw)):
        smiles =df_raw.at[i,'SMILES_raw']
        mol = Chem.MolFromSmiles(smiles)
        sd_mol =  standardize_molecule(mol)
        sd_smiles = create_sd_smiles(sd_mol)
        sd_inchi_key = mol_to_inchi_key(sd_mol)
        df_raw.at[i,'papyrus_SMILES'] = sd_smiles
        df_raw.at[i,'papyrus_inchi_key'] = sd_inchi_key

    print(f'df length after standardization: {len(df_raw)}')

    return df_raw

In [ ]:
#Check for missing inchi key

def missing_inchi(df_raw):
    smiles_nan = df_raw['papyrus_SMILES'].isna().sum()
    inchikey_nan =df_raw['papyrus_inchi_key'].isna().sum()
    print(f'DB length: {len(df_raw)},        SMILES nan: {smiles_nan},        inchi key nan: {inchikey_nan}')

    #Remove rows with missing inchikey
    df_valid_inchi= df_raw[((df_raw['papyrus_inchi_key'].notna()))]
    print('-----remove missing inchikey----')
    print(f'updated length: {len(df_valid_inchi)}')

    return df_valid_inchi

In [ ]:
#Create connecitivty inchi 

def inchi_first_part(inchi):
    return inchi.split('-')[0]

def create_connectivity_inchi(df):
    df['inchi_connectivity'] = df['papyrus_inchi_key'].apply(inchi_first_part)
    
    return df

In [ ]:
#Check for duplicates

def remove_duplicates(df):
    print(f'length: {len(df)}')
    inchi_un = df['inchi_connectivity'].nunique()
    print(f'unique_inchi: {inchi_un}')


    unique_counts = df.groupby('inchi_connectivity')['status_cns'].nunique()
    duplicates_diff_class = unique_counts[unique_counts > 1].index

    print(f'Contradicting duplicates: {len(duplicates_diff_class)}')

    #Remove duplicates
    df = df[~(df['inchi_connectivity'].isin(duplicates_diff_class))]
    print(len(df))
    print(df['inchi_connectivity'].nunique())

    df=df.drop_duplicates(subset=['inchi_connectivity'], keep="first").reset_index(drop=True)
    #df['inchi_stereo'].values_counts

    return df

In [ ]:
#Run all the functions

df_sd = standardize_workflow(cns)
df_valid = missing_inchi(df_sd)
df_connectivity_inchi = create_connectivity_inchi(df_valid)
df_no_duplicates = remove_duplicates(df_connectivity_inchi)


In [ ]:
df_no_duplicates.head()

Remove training molecules

In [ ]:
#Load training datasets

influx = pd.read_csv('../../../3_Combined/31_Combined_datasets/1_data_curation/influx/combined_influx_train_raw.csv')
efflux = pd.read_csv('../../../3_Combined/31_Combined_datasets/1_data_curation/efflux/combined_efflux_train_raw.csv')
pampa = pd.read_csv('../../../3_Combined/31_Combined_datasets/1_data_curation/pampa/combined_pampa_train_raw.csv')
bbb = pd.read_csv('../../../3_Combined/31_Combined_datasets/1_data_curation/bbb/combined_bbb_train_raw.csv')

In [ ]:
#Create mask for removing traning compounds: influx

mask = df_no_duplicates['inchi_connectivity'].isin(influx['inchi_connectivity'])
common_values_list = df_no_duplicates[mask]['inchi_connectivity'].tolist()
print(f'Common molecules {len(common_values_list)}')
cns_val = df_no_duplicates[~mask]
cns_val = cns_val.reset_index(drop=True)
print(f'After removed: {len(cns_val)}')

In [ ]:
#Create mask for removing traning compounds: efflux

mask = cns_val['inchi_connectivity'].isin(efflux['inchi_connectivity'])
common_values_list = cns_val[mask]['inchi_connectivity'].tolist()
print(f'Common molecules {len(common_values_list)}')
cns_val = cns_val[~mask]
cns_val = cns_val.reset_index(drop=True)
print(f'After removed: {len(cns_val)}')


In [ ]:
#Create mask for removing traning compounds: pampa

mask = cns_val['inchi_connectivity'].isin(pampa['inchi_connectivity'])
common_values_list = cns_val[mask]['inchi_connectivity'].tolist()
print(f'Common molecules {len(common_values_list)}')
cns_val = cns_val[~mask]
cns_val = cns_val.reset_index(drop=True)
print(f'After removed: {len(cns_val)}')

In [ ]:
#Create mask for removing traning compounds: bbb

mask = cns_val['inchi_connectivity'].isin(bbb['inchi_connectivity'])
common_values_list = cns_val[mask]['inchi_connectivity'].tolist()
print(f'Common molecules {len(common_values_list)}')
cns_val = cns_val[~mask]
cns_val = cns_val.reset_index(drop=True)
print(f'After removed: {len(cns_val)}')

cns_val.to_csv('drugbank_validation.csv')